<a href="https://colab.research.google.com/github/Anunineleaps6/Capstone_Sales/blob/main/Capstone_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [538]:
!pip install gspread oauth2client pandas numpy


In [539]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/CAPSTONE_FINAL/creds.json", scope)
client = gspread.authorize(creds)


In [540]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta


In [541]:
!pip install phonenumbers

In [542]:
import phonenumbers

def generate_phone(country, region_mapping):
    # Try to get the country code from the region_mapping first
    code = None
    for region, data in region_mapping.items():
        if country in data["countries"]:
            code = data["dial_code"].get(country)
            break

    if code:
        try:
            # Generate a random 10-digit number
            local_number = ''.join([str(random.randint(0, 9)) for _ in range(10)])
            # Attempt to parse and format using phonenumbers
            # Note: This might not create *valid* numbers, but will follow the format
            # We need a country code without the '+' for the parse function
            country_code_for_parse = code.lstrip('+')
            # We need to find a region code (like US, GB, IN) for the parse function
            # This is a simplified approach; a more robust solution would map countries to region codes
            country_region_code = ""
            for region_key, data in region_mapping.items():
                if country in data.get("countries", []):
                     # This is a simplification; need a proper country to region code mapping
                     # For now, we can try to infer from common country codes or use a default
                     # A more accurate approach would involve a comprehensive country code to region code mapping
                     # Let's use a placeholder logic or rely on the dial code if a direct mapping is complex
                     pass # Will refine this

            # A better approach might be to directly format a random number with the known dial code
            # We'll stick to generating a random 10-digit number and prefixing the dial code
            # as phonenumbers.parse requires a known region or a leading '+' which we already have

            # Let's refine the phonenumbers usage or simplify if direct generation is sufficient
            # Given the user wants the format "+[country_code]-[10_digits]", direct generation is simpler
            number = ''.join([str(random.randint(0, 9)) for _ in range(10)])
            return f"{code}-{number}"

        except Exception as e:
            print(f"Error generating phone for {country} with code {code}: {e}")
            # Fallback to a generic format if phonenumbers fails
            return f"{code}-{random.randint(1000000000, 9999999999)}"
    else:
        # Fallback for countries not in region_mapping
        return f"+00-{random.randint(1000000000, 9999999999)}"

# Now, modify the function call in the lead generation loop in cell 9212e2f3
# (This modification will be done in the next modify_cells command)

In [543]:
#LEADS
import pandas as pd
import random
import uuid
from datetime import datetime, timedelta

# ------------------------------------------
# 1. Region, Country, Dial Code Mapping (Used for phone number generation)
# ------------------------------------------
region_mapping = {
    "NA": {"countries": ["USA", "Canada"], "dial_code": {"USA": "+1", "Canada": "+1"}, "industries": ["Technology", "Healthcare", "Finance", "Retail"], "services": ["Data", "Marketing", "Cloud"]},
    "EU": {"countries": ["UK", "Germany", "France", "Netherlands", "Switzerland", "Sweden", "Israel"], "dial_code": {"UK": "+44", "Germany": "+49", "France": "+33", "Netherlands": "+31", "Switzerland": "+41", "Sweden": "+46", "Israel": "+972"}, "industries": ["Automotive", "Pharmaceuticals", "Banking", "Consulting"], "services": ["Consulting", "Engineering", "Operations"]},
    "APAC": {"countries": ["India", "Japan", "Singapore", "China", "Indonesia", "Australia", "New Zealand", "S. Korea"], "dial_code": {"India": "+91", "Japan": "+81", "Singapore": "+65", "China": "+86", "Indonesia": "+62", "Australia": "+61", "New Zealand": "+64", "S. Korea": "+82"}, "industries": ["Technology", "E-Commerce", "Manufacturing", "Finance"], "services": ["AI", "Staffing", "T&M"]},
    "ME": {"countries": ["UAE", "Saudi Arabia", "Qatar", "Bahrain"], "dial_code": {"UAE": "+971", "Saudi Arabia": "+966", "Qatar": "+974", "Bahrain": "+973"}, "industries": ["Energy", "Real Estate", "Banking"], "services": ["Data", "Consulting"]},
    "LA": {"countries": ["Brazil", "Mexico", "Argentina", "Chile", "Colombia", "Peru"], "dial_code": {"Brazil": "+55", "Mexico": "+52", "Argentina": "+54", "Chile": "+56", "Colombia": "+57", "Peru": "+51"}, "industries": ["Manufacturing", "Agriculture", "Energy"], "services": ["Operations", "Logistics"]},
    "AF": {"countries": ["S. Africa", "Mozambique", "Kenya", "Morocco"], "dial_code": {"S. Africa": "+27", "Mozambique": "+258", "Kenya": "+254", "Morocco": "+212"}, "industries": ["Telecommunications", "Mining", "Finance"], "services": ["Staffing", "Service"]}
}

# ------------------------------------------
# 2. Predefined Lead Data (for Company_Name, Country, Region, Lead_ID) - 108 Companies
# ------------------------------------------
predefined_leads = [
    # North America (20)
    ("Uber", "USA", "NA", "LDUBR-NA-001"), ("Walmart", "USA", "NA", "LDWMT-NA-002"), ("Ford Motor Co", "USA", "NA", "LDFRD-NA-003"), ("Amazon", "USA", "NA", "LDAMZ-NA-004"),
    ("Microsoft", "USA", "NA", "LDMST-NA-005"), ("Target", "USA", "NA", "LDTGT-NA-006"), ("Uber Eats", "USA", "NA", "LDUBE-NA-007"), ("CVS Health", "USA", "NA", "LDCVS-NA-008"),
    ("McKesson", "USA", "NA", "LDMCK-NA-009"), ("TD Bank Group", "Canada", "NA", "LDTDB-NA-010"), ("Shopify", "Canada", "NA", "LDSHF-NA-011"), ("Suncor Energy", "Canada", "NA", "LDSCR-NA-012"),
    ("Kaiser Permanente", "USA", "NA", "LDKPR-NA-013"), ("Wells Fargo", "USA", "NA", "LDWLF-NA-014"), ("Canadian Tire", "Canada", "NA", "LDCTC-NA-015"), ("American Tower", "USA", "NA", "LDAMT-NA-016"),
    ("Tim Hortons", "Canada", "NA", "LDTHC-NA-017"), ("Lowe's", "USA", "NA", "LDLWS-NA-018"), ("PicnicHealth", "USA", "NA", "LDPCH-NA-019"), ("Credit Saison India (USA operations)", "USA", "NA", "LDCSI-NA-020"),

    # Europe (16)
    ("Tesco", "UK", "EU", "LDTSCO-EU-021"), ("Lufthansa Group", "Germany", "EU", "LDLFT-EU-022"), ("Allianz Direct", "Germany", "EU", "LDALZ-EU-023"), ("HSBC", "UK", "EU", "LDHSBC-EU-024"),
    ("Sage Group", "UK", "EU", "LDSGE-EU-025"), ("Schneider Electric", "France", "EU", "LDSNE-EU-026"), ("Philips Healthcare", "Netherlands", "EU", "LDPHL-EU-027"), ("Vodafone", "UK", "EU", "LDVDF-EU-028"),
    ("Siemens", "Germany", "EU", "LDSMN-EU-029"), ("Nestlé", "Switzerland", "EU", "LDNST-EU-030"), ("ING Group", "Netherlands", "EU", "LDING-EU-031"), ("ABB", "Switzerland", "EU", "LDABB-EU-032"),
    ("Volvo Group", "Sweden", "EU", "LDVLV-EU-033"), ("Carrefour", "France", "EU", "LDCFR-EU-034"), ("ZF Group", "Germany", "EU", "LDZFG-EU-035"), ("Trigo", "UK", "EU", "LDTRG-EU-036"), # Assuming UK operations for Trigo

    # Asia-Pacific (14)
    ("DBS Bank", "Singapore", "APAC", "LDDBS-APAC-037"), ("Kiwibank", "New Zealand", "APAC", "LDKWB-APAC-038"), ("Reliance Retail", "India", "APAC", "LDRLR-APAC-039"), ("Tata Consultancy Services", "India", "APAC", "LDTCS-APAC-040"),
    ("SoftBank", "Japan", "APAC", "LDSFB-APAC-041"), ("Tokopedia / GoTo", "Indonesia", "APAC", "LDTKP-APAC-042"), ("Telstra", "Australia", "APAC", "LDTSL-APAC-043"), ("Westpac", "Australia", "APAC", "LDWSP-APAC-044"),
    ("infosys", "India", "APAC", "LDINF-APAC-045"), ("Grab", "Singapore", "APAC", "LDGRB-APAC-046"), ("JioMart", "India", "APAC", "LDJMR-APAC-047"), ("Toyota", "Japan", "APAC", "LDTYT-APAC-048"),
    ("Sony", "Japan", "APAC", "LDSNY-APAC-049"), ("Samsung", "S. Korea", "APAC", "LDSMS-APAC-050"),

    # Middle East (4)
    ("Saudi Aramco", "Saudi Arabia", "ME", "LDSAR-ME-051"), ("ADNOC", "UAE", "ME", "LDADC-ME-052"), ("QatarEnergy", "Qatar", "ME", "LDQTE-ME-053"), ("Investcorp", "Bahrain", "ME", "LDINV-ME-054"),

    # Latin America (10)
    ("Petrobras", "Brazil", "LA", "LDPBR-LA-055"), ("América Móvil", "Mexico", "LA", "LDAMX-LA-056"), ("MercadoLibre", "Argentina", "LA", "LDMCL-LA-057"), ("Codelco", "Chile", "LA", "LDCDL-LA-058"),
    ("Ecopetrol", "Colombia", "LA", "LDECP-LA-059"), ("Latam Airlines", "Chile", "LA", "LDLTM-LA-060"), ("Natura", "Brazil", "LA", "LDNTR-LA-061"), ("Itau Unibanco", "Brazil", "LA", "LDITU-LA-062"),
    ("Bancolombia", "Colombia", "LA", "LDBCL-LA-063"), ("Grupo Mexico", "Mexico", "LA", "LDGPM-LA-064"),

    # Africa (12)
    ("MTN Group", "S. Africa", "AF", "LDMTN-AF-065"), ("Sasol", "S. Africa", "AF", "LDSAL-AF-066"), ("Standard Bank Group", "S. Africa", "AF", "LDSBG-AF-067"), ("Vodacom", "S. Africa", "AF", "LDVDC-AF-068"),
    ("Safaricom", "Kenya", "AF", "LDSFC-AF-069"), ("KCB Group", "Kenya", "AF", "LDKCB-AF-070"), ("Attijariwafa Bank", "Morocco", "AF", "LDATJ-AF-071"), ("FirstRand", "S. Africa", "AF", "LDFRD-AF-072"),
    ("Absa Group", "S. Africa", "AF", "LDABG-AF-073"), ("Nedbank Group", "S. Africa", "AF", "LDNDB-AF-074"), ("Equity Group Holdings", "Kenya", "AF", "LDEGH-AF-075"), ("Telkom", "S. Africa", "AF", "LDTLK-AF-076"),

    # Adding more predefined companies to reach 108, maintaining the pattern and region assignment
    ("Infosys", "India", "APAC", "LDINF-APAC-077"), ("Wipro", "India", "APAC", "LDWIP-APAC-078"), ("HCL Technologies", "India", "APAC", "LDHCL-APAC-079"),
    ("Tech Mahindra", "India", "APAC", "LDTMH-APAC-080"), ("Larsen & Toubro Infotech", "India", "APAC", "LDLTI-APAC-081"), ("Mindtree", "India", "APAC", "LDMNT-APAC-082"),
    ("Mphasis", "India", "APAC", "LDMPH-APAC-083"), ("L&T Technology Services", "India", "APAC", "LDLTTS-APAC-084"), ("Persistent Systems", "India", "APAC", "LDPST-APAC-085"),
    ("Coforge", "India", "APAC", "LDCFG-APAC-086"), ("Zensar Technologies", "India", "APAC", "LDZNR-APAC-087"), ("Firstsource Solutions", "India", "APAC", "LDFRS-APAC-088"),
    ("ExlService Holdings", "India", "APAC", "LDEXL-APAC-089"), ("Genus Power Infrastructures", "India", "APAC", "LDGPI-APAC-090"), ("Cyient", "India", "APAC", "LDCYI-APAC-091"),

    ("Tata Motors", "India", "APAC", "LDTMO-APAC-092"), ("Mahindra & Mahindra", "India", "APAC", "LDMAM-APAC-093"), ("Maruti Suzuki India", "India", "APAC", "LDMSI-APAC-094"),
    ("Hyundai Motor India", "India", "APAC", "LDHMI-APAC-095"), ("Kia India", "India", "APAC", "LDKIA-APAC-096"), ("Toyota Kirloskar Motor", "India", "APAC", "LDTKM-APAC-097"),
    ("Honda Cars India", "India", "APAC", "LDHCI-APAC-098"), ("Renault India", "India", "APAC", "LDRNI-APAC-099"), ("Nissan Motor India", "India", "APAC", "LDNMI-APAC-100"),

    ("Apple", "USA", "NA", "LDAPL-NA-101"), ("Google", "USA", "NA", "LDGOO-NA-102"), ("Facebook", "USA", "NA", "LDFAB-NA-103"), ("Netflix", "USA", "NA", "LDNFL-NA-104"),
    ("Salesforce", "USA", "NA", "LDSFC-NA-105"), ("Tesla", "USA", "NA", "LDTES-NA-106"), ("IBM", "USA", "NA", "LDIBM-NA-107"), ("Intel", "USA", "NA", "LDINT-NA-108")
]


# ------------------------------------------
# 3. Generate Sales Reps (Unevenly distributed based on region size)
# ------------------------------------------
rep_names = [
    "Alice Smith", "Bob Johnson", "Charlie Lee", "Diana King", "Edward Brown",
    "Fiona Green", "George White", "Hannah Black", "Ian Gray", "Jill Lewis",
    "Kevin Martin", "Laura Scott", "Mike Turner", "Nina Adams", "Oscar Clark",
    "Paula Young", "Quinn Hall", "Rachel Allen", "Steve Wright", "Tina Baker"
] # 20 names

# Distribute 20 reps unevenly across regions (adjust numbers as needed to total 20)
# Example: NA: 4, EU: 4, APAC: 5, ME: 2, LA: 3, AF: 2
region_rep_distribution = {"NA": 4, "EU": 4, "APAC": 5, "ME": 2, "LA": 3, "AF": 2}

rep_list = []
rep_name_index = 0
for region, count in region_rep_distribution.items():
    for i in range(count):
        full_name = rep_names[rep_name_index]
        initials = "".join([part[0] for part in full_name.split()]).upper()
        rep_id = f"{initials}_{region}"
        rep_list.append({"rep_id": rep_id, "rep_name": full_name, "region": region})
        rep_name_index += 1

# Convert to DataFrame
rep_df = pd.DataFrame(rep_list)

# ------------------------------------------
# 4. Generate Leads DataFrame
# ------------------------------------------
def get_company_email(name):
    # Basic email domain generation
    domain = name.lower().replace(" ", "").replace("&", "").replace(".", "").replace("/", "").replace("(", "").replace(")", "") + ".com"
    return domain

# Use the generate_phone function defined in the previous cell (ad0fa588)

def generate_timestamp(start_days_ago=730): # Up to 2 years ago
    created = datetime.now() - timedelta(days=random.randint(0, start_days_ago), minutes=random.randint(0, 1440), seconds=random.randint(0, 86400))
    updated = created + timedelta(minutes=random.randint(10, 10000)) # Updated is after created
    # Ensure Last_Updated is not earlier than Created_Date, add a buffer if needed
    if updated < created:
         updated = created + timedelta(minutes=random.randint(10, 60)) # Ensure at least 10 minutes difference

    return created.strftime("%Y-%m-%d %H:%M:%S"), updated.strftime("%Y-%m-%d %H:%M:%S")

def get_company_initials(name):
    return "".join([w[0] for w in name.split()]).upper()

# Define service categories and industries based on region_mapping for random lead generation
all_service_categories = list(set([service for region_data in region_mapping.values() for service in region_data.get('services', [])]))
all_industries = list(set([industry for region_data in region_mapping.values() for industry in region_data.get('industries', [])]))


# Generate a list of more descriptive company name parts for random leads
company_name_parts = {
    "adjectives": ["Global", "Strategic", "Innovative", "Dynamic", "Future", "Leading", "Creative", "Advanced"],
    "nouns": ["Solutions", "Systems", "Analytics", "Consulting", "Tech", "Ventures", "Partners", "Industries"]
}

def generate_random_company_name(adjectives, nouns):
    return f"{random.choice(adjectives)}{random.choice(nouns)}"

def get_abbreviated_company_name(name):
    # Simple abbreviation logic
    words = name.split()
    if len(words) > 1:
        return "".join([w[:2] for w in words]).upper() # Use first two letters of each word
    else:
        return name[:4].upper() # Use first four letters if single word


leads_data = []

# Add predefined leads
for company_name, country, region, lead_id in predefined_leads:
    company_email_domain = get_company_email(company_name)
    company_id = f"CMP_{get_company_initials(company_name)}"

    created_date, last_updated = generate_timestamp()
    deal_source = random.choices(["Inbound", "Outbound"], weights=[0.2, 0.8])[0]
    phone = generate_phone(country, region_mapping) # Use the new generate_phone function

    # Select a rep from same region
    region_reps = rep_df[rep_df["region"] == region]
    if not region_reps.empty:
        rep_row = region_reps.sample(n=1).iloc[0]
        assigned_rep_id = rep_row["rep_id"]
        rep_name = rep_row["rep_name"] # Keep rep_name for potential use
    else:
        # Handle case where no rep is available for the region
        assigned_rep_id = "Unassigned"
        rep_name = "Unassigned"

    lead_email = f"contact_{random.randint(1,100)}@{company_email_domain}"

    # Assign a service category and industry for predefined leads based on their country's typical services/industries
    # Find the region for the current country
    current_region = None
    for r, data in region_mapping.items():
        if country in data["countries"]:
            current_region = r
            break

    service_category = random.choice(region_mapping.get(current_region, {}).get('services', all_service_categories)) if current_region and region_mapping.get(current_region, {}).get('services') else random.choice(all_service_categories)
    company_industry = random.choice(region_mapping.get(current_region, {}).get('industries', all_industries)) if current_region and region_mapping.get(current_region, {}).get('industries') else random.choice(all_industries)


    leads_data.append({
        "Lead_ID": lead_id,
        "Lead_Name": f"{company_name} Lead",
        "Company_Name": company_name,
        "Source": deal_source,
        "Region_ID": region,
        "Rep_ID": assigned_rep_id,
        "Created_Date": created_date,
        "Current_Stage": random.choice(["Lead Captured", "Qualified", "Contacted", "Responded", "Demo Scheduled", "Demo", "Demo Completed", "Proposal Sent", "Proposal", "Negotiation", "Deal Won", "Deal Lost"]), # Include all possible stages
        "Email": lead_email,
        "Phone": phone,
        "Priority": random.choice(["High", "Medium", "Low"]),
        "Last_Updated": last_updated,
        "Deal_Source": deal_source,
        "Company_ID": company_id,
        "Engagement_Level": random.choice(["High", "Medium", "Low"]),
        "Follow_Up_Threshold": random.choice([3, 5, 7, 10]),
        "Project_Term": random.choice(["0-6 months", "6-12 months", "12-24 months", ">24 months"]),
        "Budget_Tier": random.choice(["20-50L", "50-100L", "100L+"]),
        "Country": country,
        "Company_Industry": company_industry,
        "Service_Category": service_category
    })

# Generate additional random leads to reach 5,000 (changed from 10,000)
num_predefined_leads = len(predefined_leads)
num_additional_leads = 5000 - num_predefined_leads # Changed target number of leads

# Get lists of all possible values for random generation
all_countries = [country for region_data in region_mapping.values() for country in region_data["countries"]]
all_regions = list(region_mapping.keys())

for i in range(num_additional_leads):
    company_name = generate_random_company_name(company_name_parts["adjectives"], company_name_parts["nouns"])
    country = random.choice(all_countries)
    # Find the region for the selected country using region_mapping
    region = next(r for r, data in region_mapping.items() if country in data["countries"])


    # Generate Lead ID in the desired format for random leads
    company_abbreviation = get_abbreviated_company_name(company_name)
    lead_id = f"LD{company_abbreviation}-{region}-{i+1:04}" # Adjusted padding for 5000 leads

    company_email_domain = get_company_email(company_name)
    company_id = f"CMP_{get_company_initials(company_name)}_{i+1:04}"

    created_date, last_updated = generate_timestamp()
    deal_source = random.choices(["Inbound", "Outbound"], weights=[0.2, 0.8])[0]
    phone = generate_phone(country, region_mapping) # Use the new generate_phone function

    # Select a rep from same region
    region_reps = rep_df[rep_df["region"] == region]
    if not region_reps.empty:
        rep_row = region_reps.sample(n=1).iloc[0]
        assigned_rep_id = rep_row["rep_id"]
        rep_name = rep_row["rep_name"]
    else:
        # Handle case where no rep is available for the region
        assigned_rep_id = "Unassigned"
        rep_name = "Unassigned"

    lead_email = f"contact_{random.randint(1,100)}@{company_email_domain}"

    # Assign a service category and industry for random leads based on the selected country's focus
    # Find the region for the current country
    current_region = None
    for r, data in region_mapping.items():
        if country in data["countries"]:
            current_region = r
            break

    service_category = random.choice(region_mapping.get(current_region, {}).get('services', all_service_categories)) if current_region and region_mapping.get(current_region, {}).get('services') else random.choice(all_service_categories)
    company_industry = random.choice(region_mapping.get(current_region, {}).get('industries', all_industries)) if current_region and region_mapping.get(current_region, {}).get('industries') else random.choice(all_industries)


    # Generate other random data, potentially influenced by service_category if needed
    # (Currently, the random generation is independent of service_category,
    #  but can be made dependent here if specific correlations are desired)
    engagement_level = random.choice(["High", "Medium", "Low"])
    follow_up_threshold = random.choice([3, 5, 7, 10])
    project_term = random.choice(["0-6 months", "6-12 months", "12-24 months", ">24 months"])
    budget_tier = random.choice(["20-50L", "50-100L", "100L+"])
    current_stage = random.choice(["Lead Captured", "Qualified", "Contacted", "Responded", "Demo Scheduled", "Demo", "Demo Completed", "Proposal Sent", "Proposal", "Negotiation", "Deal Won", "Deal Lost"]) # Include all possible stages
    priority = random.choice(["High", "Medium", "Low"])


    leads_data.append({
        "Lead_ID": lead_id, # Use the new formatted Lead ID
        "Lead_Name": f"{company_name} Lead",
        "Company_Name": company_name,
        "Source": deal_source, # Changed from "Lead Source" to "Source" to match requested column names
        "Region_ID": region,
        "Rep_ID": assigned_rep_id,
        "Created_Date": created_date,
        "Current_Stage": current_stage,
        "Email": lead_email,
        "Phone": phone,
        "Priority": priority,
        "Last_Updated": last_updated,
        "Deal_Source": deal_source,
        "Company_ID": company_id,
        "Engagement_Level": engagement_level,
        "Follow_Up_Threshold": follow_up_threshold,
        "Project_Term": project_term,
        "Budget_Tier": budget_tier,
        "Country": country,
        "Company_Industry": company_industry,
        "Service_Category": service_category
    })


df_lead = pd.DataFrame(leads_data)

# Reorder columns to match the requested order plus new columns
df_lead = df_lead[[
    "Lead_ID", "Lead_Name", "Company_Name", "Source", "Region_ID", "Rep_ID",
    "Created_Date", "Current_Stage", "Email", "Phone", "Priority", "Last_Updated",
    "Deal_Source", "Company_ID", "Engagement_Level", "Follow_Up_Threshold",
    "Project_Term", "Budget_Tier", "Country", "Company_Industry", "Service_Category"
]]


# After generating the initial df_lead, update the 'Current_Stage'
# to reflect the latest stage from the generated df_stage_history.
# This part needs to be executed after df_stage_history is generated in the next cell.
# This logic will be moved to a subsequent cell or integrated carefully if possible.


# ------------------------------------------
# 5. Export or View
# ------------------------------------------
# Save to CSV
# df_lead.to_csv("crm_leads_generated.csv", index=False)

# Display sample
display(df_lead.head())
display(df_lead.tail())
print(f"Generated {len(df_lead)} leads.")

,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Last_Updated,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category
0,LDUBR-NA-001,Uber Lead,Uber,Outbound,NA,DK_NA,2024-04-24 21:55:13,Proposal Sent,contact_58@uber.com,+1-1619817333,...,2024-04-26 05:52:13,Outbound,CMP_U,Low,7,0-6 months,50-100L,USA,Retail,Data
1,LDWMT-NA-002,Walmart Lead,Walmart,Outbound,NA,BJ_NA,2024-04-26 03:53:41,Proposal,contact_3@walmart.com,+1-4536118013,...,2024-04-28 18:11:41,Outbound,CMP_W,High,3,>24 months,50-100L,USA,Finance,Data
2,LDFRD-NA-003,Ford Motor Co Lead,Ford Motor Co,Inbound,NA,CL_NA,2024-04-27 06:06:54,Demo Scheduled,contact_83@fordmotorco.com,+1-9959636818,...,2024-05-01 11:57:54,Inbound,CMP_FMC,Medium,5,12-24 months,20-50L,USA,Technology,Marketing
3,LDAMZ-NA-004,Amazon Lead,Amazon,Outbound,NA,BJ_NA,2024-06-15 20:09:24,Lead Captured,contact_57@amazon.com,+1-7746385663,...,2024-06-17 04:20:24,Outbound,CMP_A,Medium,5,0-6 months,20-50L,USA,Technology,Data
4,LDMST-NA-005,Microsoft Lead,Microsoft,Outbound,NA,BJ_NA,2023-12-12 06:15:25,Demo Completed,contact_15@microsoft.com,+1-6596503700,...,2023-12-13 07:47:25,Outbound,CMP_M,Medium,10,0-6 months,20-50L,USA,Technology,Data


,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Last_Updated,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category
4995,LDSTRA-ME-4888,StrategicIndustries Lead,StrategicIndustries,Outbound,ME,OC_ME,2025-01-25 13:58:40,Negotiation,contact_34@strategicindustries.com,+973-6083148014,...,2025-01-28 12:45:40,Outbound,CMP_S_4888,Low,7,>24 months,100L+,Bahrain,Real Estate,Data
4996,LDINNO-EU-4889,InnovativeIndustries Lead,InnovativeIndustries,Outbound,EU,EB_EU,2023-10-29 18:43:36,Demo,contact_2@innovativeindustries.com,+44-7150916889,...,2023-11-04 07:09:36,Outbound,CMP_I_4889,Low,5,>24 months,100L+,UK,Pharmaceuticals,Consulting
4997,LDADVA-LA-4890,AdvancedConsulting Lead,AdvancedConsulting,Outbound,LA,QH_LA,2024-08-11 22:42:41,Demo Scheduled,contact_100@advancedconsulting.com,+52-0031194698,...,2024-08-16 01:48:41,Outbound,CMP_A_4890,Medium,10,12-24 months,100L+,Mexico,Agriculture,Logistics
4998,LDFUTU-EU-4891,FutureAnalytics Lead,FutureAnalytics,Outbound,EU,HB_EU,2024-06-25 10:55:51,Lead Captured,contact_21@futureanalytics.com,+44-6063960473,...,2024-06-26 11:12:51,Outbound,CMP_F_4891,Low,3,0-6 months,50-100L,UK,Banking,Engineering
4999,LDFUTU-LA-4892,FuturePartners Lead,FuturePartners,Outbound,LA,PY_LA,2025-02-04 19:32:41,Proposal,contact_8@futurepartners.com,+56-0027411837,...,2025-02-06 22:41:41,Outbound,CMP_F_4892,High,3,12-24 months,20-50L,Chile,Agriculture,Operations


Generated 5000 leads.


In [544]:
#REGION
import pandas as pd
import numpy as np
import random

# Use unique Region_IDs from df_lead
region_ids = df_lead['Region_ID'].unique().tolist()
timezones = ["IST", "EST", "PST", "CET", "GMT"]
countries_in_leads = df_lead['Country'].unique().tolist() # Get countries from df_lead

region_data = []

# Define value mappings for company size for target calculation
company_size_value = {
    "Start-up": 1,
    "Small business": 5,
    "Medium Enterprise": 10,
    "Large Enterprise": 20
}

for region_id in region_ids:
    # Select a country that belongs to this region based on region_mapping
    country = random.choice([c for r, data in region_mapping.items() if r == region_id for c in data['countries'] if c in countries_in_leads])

    # Find reps assigned to this region
    reps_in_region = rep_df[rep_df['region'] == region_id]['rep_name'].tolist()
    region_head = random.choice(reps_in_region) if reps_in_region else "Unassigned"

    # Calculate Region_Target based on companies in the region
    companies_in_region = df_company[df_company['Country'].apply(lambda x: x in region_mapping.get(region_id, {}).get('countries', []))]

    # Calculate a score for each company based on size, ownership, funding, and interest area (simplified)
    # You can adjust weights and factors based on your desired logic
    company_scores = companies_in_region.apply(
        lambda row: company_size_value.get(row['Company_Size'], 0) *
                    (2 if row['Company_Ownership'] == 'Private Sector' else 1) *
                    (2 if row['Company_Funding'] in ['Series B', 'IPO'] else 1) *
                    (2 if row['Interest_Area'] in region_mapping.get(region_id, {}).get('services', []) else 1),
        axis=1
    )

    # Aggregate scores to get a raw region target
    raw_region_target = company_scores.sum()

    # Scale the raw target to a reasonable range (adjust scaling factor as needed)
    # Example: Scale by 100 to get targets in the thousands/millions
    region_target = round(raw_region_target * 500, 2) # Adjusted scaling factor

    region_data.append({
        "Region_ID": region_id,
        "Region_Name": f"Region_{region_id}",
        "Region_Head": region_head,
        "Timezone": random.choice(timezones),
        "Region_Target": region_target
    })

df_region = pd.DataFrame(region_data)
display(df_region.head())

,Region_ID,Region_Name,Region_Head,Timezone,Region_Target
0,NA,Region_NA,Bob Johnson,GMT,5278000
1,EU,Region_EU,George White,CET,17610000
2,APAC,Region_APAC,Kevin Martin,GMT,20513000
3,ME,Region_ME,Nina Adams,IST,10491000
4,LA,Region_LA,Paula Young,IST,14594000


In [545]:
#SALES_REP
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Use unique Rep_IDs and their corresponding regions from df_lead
sales_rep_data = []
added_rep_ids = set()

for index, row in df_lead.iterrows():
    rep_id = row['Rep_ID']
    region_id = row['Region_ID']
    # Add rep data only if not already added
    if rep_id not in added_rep_ids:
        # Find the rep name from the rep_df based on Rep_ID
        rep_info = rep_df[rep_df['rep_id'] == rep_id]
        rep_name = rep_info['rep_name'].iloc[0] if not rep_info.empty else f"Rep_{rep_id}"


        sales_rep_data.append({
            "Rep_ID": rep_id,
            "Rep_Name": rep_name, # Use name from rep_df
            "Email": f"{rep_id.lower()}@crmcorp.com", # Generate email based on Rep_ID
            "Phone": f"+91-9{random.randint(100000000,999999999)}", # Keep existing phone generation for now
            "Region_ID": region_id,
            "Join_Date": datetime(2020, 1, 1) + timedelta(days=random.randint(0, 1000)),
            "Current_Status": random.choice(["Active", "Inactive", "On Leave"]),
            "Performance_Rating": round(random.uniform(1.0, 5.0), 1),
            "Total_Deals_Closed": random.randint(0, 500),
            "Avg_conversion_time": round(random.uniform(3.0, 30.0), 2),
            "Win_rate": round(random.uniform(0.1, 0.9), 2)
        })
        added_rep_ids.add(rep_id)

# Add any reps from rep_df who are not assigned to any lead in df_lead
reps_not_in_leads = rep_df[~rep_df['rep_id'].isin(added_rep_ids)]
for index, rep_row in reps_not_in_leads.iterrows():
    rep_id = rep_row['rep_id']
    region_id = rep_row['region']
    rep_name = rep_row['rep_name']

    sales_rep_data.append({
        "Rep_ID": rep_id,
        "Rep_Name": rep_name,
        "Email": f"{rep_id.lower()}@crmcorp.com",
        "Phone": f"+91-9{random.randint(100000000,999999999)}",
        "Region_ID": region_id,
        "Join_Date": datetime(2020, 1, 1) + timedelta(days=random.randint(0, 1000)),
        "Current_Status": random.choice(["Active", "Inactive", "On Leave"]),
        "Performance_Rating": round(random.uniform(1.0, 5.0), 1),
        "Total_Deals_Closed": random.randint(0, 500),
        "Avg_conversion_time": round(random.uniform(3.0, 30.0), 2),
        "Win_rate": round(random.uniform(0.1, 0.9), 2)
    })


df_sales_rep = pd.DataFrame(sales_rep_data)
display(df_sales_rep.head())
display(df_sales_rep.tail())
print(f"Generated {len(df_sales_rep)} sales reps.")

,Rep_ID,Rep_Name,Email,Phone,Region_ID,Join_Date,Current_Status,Performance_Rating,Total_Deals_Closed,Avg_conversion_time,Win_rate
0,DK_NA,Diana King,dk_na@crmcorp.com,+91-9831185425,NA,2020-08-21,Inactive,2.0,328,5.44,0.63
1,BJ_NA,Bob Johnson,bj_na@crmcorp.com,+91-9435306919,NA,2022-05-24,On Leave,4.0,332,5.31,0.49
2,CL_NA,Charlie Lee,cl_na@crmcorp.com,+91-9900128761,NA,2022-02-25,Active,3.2,119,19.96,0.53
3,EB_EU,Edward Brown,eb_eu@crmcorp.com,+91-9162209105,EU,2022-04-03,Inactive,2.5,46,25.98,0.63
4,FG_EU,Fiona Green,fg_eu@crmcorp.com,+91-9799913163,EU,2021-12-07,Inactive,2.7,191,19.69,0.69


,Rep_ID,Rep_Name,Email,Phone,Region_ID,Join_Date,Current_Status,Performance_Rating,Total_Deals_Closed,Avg_conversion_time,Win_rate
15,QH_LA,Quinn Hall,qh_la@crmcorp.com,+91-9421284536,LA,2020-12-06,On Leave,3.9,41,15.61,0.36
16,TB_AF,Tina Baker,tb_af@crmcorp.com,+91-9474421586,AF,2022-05-05,On Leave,1.0,196,14.79,0.61
17,SW_AF,Steve Wright,sw_af@crmcorp.com,+91-9143709784,AF,2021-07-23,Inactive,3.9,193,12.08,0.78
18,AS_NA,Alice Smith,as_na@crmcorp.com,+91-9639166340,NA,2021-07-07,Inactive,3.3,437,18.19,0.19
19,HB_EU,Hannah Black,hb_eu@crmcorp.com,+91-9127713537,EU,2021-03-07,On Leave,4.1,5,13.02,0.49


Generated 20 sales reps.


In [546]:
#COMPANIES
import pandas as pd
import numpy as np
import random

industries = ["Technology", "Healthcare", "Finance", "Manufacturing", "Retail", "E-Commerce", "Green Tech", "Mobility", "Real Estate", "Banking", "Logistics", "Housing", "Energy"] # Expanded industries
sizes = ["Start-up", "Small business", "Medium Enterprise", "Large Enterprise"]
ownerships = ["Public Sector", "Private Sector", "Joint Venture", "Franchise"]
funding = ["Bootstrapped", "Seed", "Series A", "Series B", "IPO"]
interest_areas = ["Data", "Marketing", "Cloud", "Design", "HR", "AI", "Consulting", "Engineering", "Operations", "Staffing", "T&M"] # Expanded interest areas

company_data = []
added_company_ids = set()

# Add companies from df_lead
for index, row in df_lead.iterrows():
    company_id = row['Company_ID']
    company_name = row['Company_Name']
    country = row['Country']
    # Add company data only if not already added
    if company_id not in added_company_ids:
        # Attempt to get industry and service from region_mapping if available, otherwise use random
        # Find the country info in the region_mapping dictionary
        current_region_data = next((data for region, data in region_mapping.items() if country in data["countries"]), None)

        company_industry = random.choice(current_region_data.get('industries', industries)) if current_region_data and current_region_data.get('industries') else random.choice(industries)
        interest_area = random.choice(current_region_data.get('services', interest_areas)) if current_region_data and current_region_data.get('services') else random.choice(interest_areas)

        company_data.append({
            "Company_ID": company_id,
            "Company_Name": company_name,
            "Company_Industry": company_industry, # Use industry from region_mapping or random
            "Company_Size": random.choice(sizes),
            "Company_Ownership": random.choice(ownerships),
            "Company_Funding": random.choice(funding),
            "Country": country, # Use country from df_lead
            "Interest_Area": interest_area # Use service from region_mapping or random
        })
        added_company_ids.add(company_id)

# Generate additional companies if needed (optional, based on whether df_lead has all companies)
# If df_lead contains all the companies you need for df_company, you might not need this part.
# Assuming df_lead has the comprehensive list of companies you want in df_company.
# (No additional generation needed as df_lead now contains 10000 companies)

df_company = pd.DataFrame(company_data)
display(df_company.head())
display(df_company.tail())
print(f"Generated {len(df_company)} companies.")

,Company_ID,Company_Name,Company_Industry,Company_Size,Company_Ownership,Company_Funding,Country,Interest_Area
0,CMP_U,Uber,Technology,Small business,Public Sector,Bootstrapped,USA,Cloud
1,CMP_W,Walmart,Technology,Medium Enterprise,Public Sector,Seed,USA,Marketing
2,CMP_FMC,Ford Motor Co,Technology,Start-up,Private Sector,Series B,USA,Data
3,CMP_A,Amazon,Healthcare,Large Enterprise,Franchise,Bootstrapped,USA,Cloud
4,CMP_M,Microsoft,Retail,Small business,Joint Venture,Seed,USA,Cloud


,Company_ID,Company_Name,Company_Industry,Company_Size,Company_Ownership,Company_Funding,Country,Interest_Area
4958,CMP_S_4888,StrategicIndustries,Real Estate,Start-up,Franchise,Series A,Bahrain,Data
4959,CMP_I_4889,InnovativeIndustries,Banking,Small business,Joint Venture,Seed,UK,Consulting
4960,CMP_A_4890,AdvancedConsulting,Energy,Start-up,Private Sector,Series A,Mexico,Operations
4961,CMP_F_4891,FutureAnalytics,Banking,Small business,Joint Venture,Series B,UK,Engineering
4962,CMP_F_4892,FuturePartners,Manufacturing,Large Enterprise,Public Sector,Seed,Chile,Logistics


Generated 4963 companies.


In [547]:
#STAGE HISTORY
import pandas as pd
import random
from datetime import datetime, timedelta

# Define stages in the desired funnel order based on user provided ranges
funnel_stages = [
    "Lead Captured",
    "Qualified",
    "Contacted",
    "Responded",
    "Demo Scheduled",
    "Demo",
    "Demo Completed",
    "Proposal Sent",
    "Proposal",
    "Negotiation",
    "Deal Won",
    "Deal Lost"
]

stage_history_data = []
history_id = 1

# Define realistic time ranges for progression between stages (in days)
# Using the specific ranges provided by the user
stage_time_ranges = {
    "Lead Captured": (2, 7),        # 2–7 days to qualification
    "Qualified": (4, 9),            # 4–9 days to contact
    "Contacted": (5, 12),           # 5–12 days to responded/demo
    "Responded": (3, 6),            # 3–6 days to demo scheduled
    "Demo Scheduled": (4, 8),       # 4–8 days to demo
    "Demo": (2, 3),                 # 2–3 days to complete demo
    "Demo Completed": (4, 9),       # 4–9 days to proposal
    "Proposal Sent": (5, 11),       # 5–11 days to proposal
    "Proposal": (6, 13),            # 6–13 days to negotiation
    "Negotiation": (7, 15)          # 7–15 days to close
}

def add_random_time_of_day(date):
    """Adds a random time of day (hour, minute, second) to a date."""
    # Ensure 'date' is a datetime object before adding timedelta
    if isinstance(date, datetime):
        return date + timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))
    elif isinstance(date, pd.Timestamp):
         return date + timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))
    else:
        # If it's a date object, convert to datetime before adding time
        return datetime(date.year, date.month, date.day) + timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))


for index, lead in df_lead.iterrows():
    lead_id = lead.Lead_ID
    final_stage = lead['Current_Stage']

    # Ensure the final_stage is in funnel_stages before proceeding
    if final_stage not in funnel_stages:
        print(f"Warning: Lead {lead.Lead_ID} has a final stage '{final_stage}' not in funnel_stages. Assigning 'Lead Captured'.")
        final_stage = "Lead Captured" # Assign a default stage if unrecognized

    # Filter the funnel stages to include only stages up to and including the final stage
    lead_stages_in_funnel = [stage for stage in funnel_stages if funnel_stages.index(stage) <= funnel_stages.index(final_stage)]

    # Ensure the stages follow the strict funnel order up to the final stage
    ordered_stages = lead_stages_in_funnel

    start_date = datetime.strptime(lead.Created_Date, "%Y-%m-%d %H:%M:%S")
    last_updated_date = datetime.strptime(lead.Last_Updated, "%Y-%m-%d %H:%M:%S")
    current_timestamp = start_date

    # Determine the number of transitions for this lead
    num_transitions = len(ordered_stages) - 1

    # Generate random days for each transition based on stage_time_ranges
    # Store these as a list of day increments
    daily_increments = []
    if num_transitions > 0:
        # Calculate the minimum total days required for this lead's stages
        min_total_days_required = sum(stage_time_ranges.get(ordered_stages[i], (1, 1))[0] for i in range(num_transitions))

        # Calculate the total available days between created and last updated dates
        total_available_days = (last_updated_date - start_date).days

        # Adjust available days to be at least the minimum required days, plus a buffer
        # If total_available_days is less than min_total_days_required, adjust the increments later
        adjusted_available_days = max(total_available_days, min_total_days_required)

        # Generate initial random increments within the stage_time_ranges
        initial_increments = []
        for i in range(num_transitions):
            prev_stage = ordered_stages[i]
            min_days, max_days = stage_time_ranges.get(prev_stage, (1, 1))
            # Ensure minimum increment is at least 1 day
            initial_increments.append(random.randint(max(1, min_days), max(1, max_days)))

        # Calculate the sum of initial increments
        sum_initial_increments = sum(initial_increments)

        # Adjust increments if the sum of initial increments doesn't match the adjusted_available_days
        # or if the total available days are less than the minimum required
        if sum_initial_increments != adjusted_available_days or total_available_days < min_total_days_required:
             # Calculate the difference that needs to be added or subtracted
             difference = adjusted_available_days - sum_initial_increments

             # Distribute the difference among the transitions
             if num_transitions > 0:
                  avg_diff_per_transition = difference // num_transitions
                  remainder = difference % num_transitions

                  daily_increments = [inc + avg_diff_per_transition for inc in initial_increments]
                  # Distribute the remainder
                  for i in range(abs(remainder)):
                      if remainder > 0:
                          daily_increments[i] += 1
                      else:
                          daily_increments[i] -= 1 # This might make increments less than min_days, handle below


                  # Ensure each increment is at least 1 day and respects the original min/max bounds as much as possible
                  for i in range(num_transitions):
                      prev_stage = ordered_stages[i]
                      min_days, max_days = stage_time_ranges.get(prev_stage, (1, 1))
                      daily_increments[i] = max(1, daily_increments[i]) # Ensure at least 1 day
                      # Optional: Clamp to original max_days if needed, but distributing difference might exceed
                      # daily_increments[i] = min(daily_increments[i], max_days) # This might break sum = adjusted_available_days


             else:
                 # If only one stage, no transitions
                 daily_increments = []

        else:
            daily_increments = initial_increments


    # Generate timestamps based on calculated daily increments
    current_timestamp = start_date
    for seq, stage in enumerate(ordered_stages, start=1):
        if seq == 1:
            # First stage timestamp is the Created_Date timestamp with random time of day
            # Pass the datetime object directly to add_random_time_of_day
            timestamp = add_random_time_of_day(start_date)
            # Ensure the generated timestamp is not before the original created_date if original had time
            if timestamp < start_date:
                 timestamp = start_date + timedelta(seconds=random.randint(1, 60)) # Ensure strictly after if on same day


        else:
            # Add the calculated days for the transition and a random time of day
            days_increment = daily_increments[seq-2]
            # Pass the datetime object directly to add_random_time_of_day
            timestamp = add_random_time_of_day(current_timestamp + timedelta(days=days_increment))

            # Ensure the timestamp is strictly increasing from the previous stage's timestamp
            while timestamp <= current_timestamp:
                 timestamp = current_timestamp + timedelta(seconds=random.randint(1, 60))


        # For the last stage, ensure its timestamp is exactly last_updated_date with preserved time
        if seq == len(ordered_stages):
             timestamp = last_updated_date
             # Ensure the last stage timestamp is not before the previous stage timestamp
             if timestamp <= current_timestamp:
                  timestamp = current_timestamp + timedelta(seconds=random.randint(1, 60)) # Add a small increment if needed


        stage_history_data.append({
            "History_ID": history_id,
            "Lead_ID": lead_id,
            "Stage": stage,
            "Timestamp": timestamp, # Store as datetime for now
            "Rep_ID": lead.Rep_ID,
            "Stage_Sequence": seq,
            "Notes": f"Notes for stage {stage} for Lead {lead_id}"
        })
        history_id += 1
        current_timestamp = timestamp # Update current_timestamp for the next iteration


df_stage_history = pd.DataFrame(stage_history_data)

# Ensure 'Timestamp' is datetime objects (should be already, but good practice)
df_stage_history['Timestamp'] = pd.to_datetime(df_stage_history['Timestamp'])

# Sort by Lead_ID and Timestamp for correct time difference calculation and display order
df_stage_history = df_stage_history.sort_values(by=['Lead_ID', 'Timestamp']).reset_index(drop=True)

# Calculate Stage_Engagement (rounded to nearest day and as a number)
stage_engagement_days = []

# Group by Lead_ID to calculate the time difference for each lead's stages
for lead_id, group in df_stage_history.groupby('Lead_ID'):
    # Sort group by timestamp to ensure correct order for diff calculation
    group = group.sort_values(by='Timestamp').reset_index(drop=True)
    for i in range(len(group)):
        if i == 0:
            # First stage entry for the lead has 0 days engagement
            days = 0
        else:
            # For all subsequent stages, calculate difference with the previous stage
            time_diff = group.iloc[i]['Timestamp'] - group.iloc[i-1]['Timestamp']
            # Calculate days, round to the nearest whole number, ensure non-negative
            days = round(time_diff.total_seconds() / (24 * 3600))
            days = max(days, 1) # Ensure non-negative and at least 1 day for subsequent stages

        stage_engagement_days.append(days)

# Add the calculated 'Stage_Engagement (in Days)' column
df_stage_history['Stage_Engagement (in Days)'] = stage_engagement_days

# Display samples of the generated DataFrame
display(df_stage_history.head())
display(df_stage_history.tail())

print(f"Generated {len(df_stage_history)} stage history entries.")

# After generating the df_stage_history, update the 'Current_Stage' in df_lead
# to reflect the latest stage from the generated df_stage_history.
# This part is moved here to ensure df_stage_history is available.

# Find the latest stage for each lead from df_stage_history (based on maximum Timestamp)
# Ensure df_stage_history is sorted by Lead_ID and Timestamp first to correctly identify the latest stage
df_stage_history = df_stage_history.sort_values(by=['Lead_ID', 'Timestamp'])

# Get the index of the last stage entry for each lead
latest_stage_indices = df_stage_history.groupby('Lead_ID')['Timestamp'].idxmax()

# Select the rows corresponding to the latest stages
latest_stages = df_stage_history.loc[latest_stage_indices][['Lead_ID', 'Stage']]
latest_stages.rename(columns={'Stage': 'Actual_Current_Stage'}, inplace=True)

# Merge with df_lead to update 'Current_Stage'
# Use a left merge to keep all leads in df_lead
df_lead = df_lead.merge(latest_stages, on='Lead_ID', how='left', suffixes=('', '_latest_history'))

# Update 'Current_Stage' in df_lead with the 'Actual_Current_Stage' from history if available
# If a lead exists in df_lead but not in df_stage_history (shouldn't happen with current logic but as a safeguard)
# or if the merge didn't find a match for some reason, keep the original 'Current_Stage'.
df_lead['Current_Stage'] = df_lead['Actual_Current_Stage'].fillna(df_lead['Current_Stage'])


# Drop the temporary 'Actual_Current_Stage' column
df_lead = df_lead.drop(columns=['Actual_Current_Stage'])

# Display samples of the updated df_lead
print("\nUpdated df_lead with latest stage information:")
display(df_lead.head())
display(df_lead.tail())

,History_ID,Lead_ID,Stage,Timestamp,Rep_ID,Stage_Sequence,Notes,Stage_Engagement (in Days)
0,218,LDABB-EU-032,Lead Captured,2024-04-14 10:59:21,FG_EU,1,Notes for stage Lead Captured for Lead LDABB-E...,0
1,493,LDABG-AF-073,Lead Captured,2024-05-17 14:10:40,SW_AF,1,Notes for stage Lead Captured for Lead LDABG-A...,0
2,494,LDABG-AF-073,Qualified,2024-05-23 03:47:35,SW_AF,2,Notes for stage Qualified for Lead LDABG-AF-073,6
3,495,LDABG-AF-073,Contacted,2024-05-28 04:48:50,SW_AF,3,Notes for stage Contacted for Lead LDABG-AF-073,5
4,496,LDABG-AF-073,Responded,2024-06-02 04:17:27,SW_AF,4,Notes for stage Responded for Lead LDABG-AF-073,5


,History_ID,Lead_ID,Stage,Timestamp,Rep_ID,Stage_Sequence,Notes,Stage_Engagement (in Days)
32154,575,LDZNR-APAC-087,Contacted,2025-02-26 17:32:31,LS_APAC,3,Notes for stage Contacted for Lead LDZNR-APAC-087,7
32155,576,LDZNR-APAC-087,Responded,2025-03-03 06:31:17,LS_APAC,4,Notes for stage Responded for Lead LDZNR-APAC-087,5
32156,577,LDZNR-APAC-087,Demo Scheduled,2025-03-06 18:40:29,LS_APAC,5,Notes for stage Demo Scheduled for Lead LDZNR-...,4
32157,578,LDZNR-APAC-087,Demo,2025-03-13 17:12:02,LS_APAC,6,Notes for stage Demo for Lead LDZNR-APAC-087,7
32158,579,LDZNR-APAC-087,Demo Completed,2025-03-13 17:12:19,LS_APAC,7,Notes for stage Demo Completed for Lead LDZNR-...,1


Generated 32159 stage history entries.

Updated df_lead with latest stage information:


,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Last_Updated,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category
0,LDUBR-NA-001,Uber Lead,Uber,Outbound,NA,DK_NA,2024-04-24 21:55:13,Proposal Sent,contact_58@uber.com,+1-1619817333,...,2024-04-26 05:52:13,Outbound,CMP_U,Low,7,0-6 months,50-100L,USA,Retail,Data
1,LDWMT-NA-002,Walmart Lead,Walmart,Outbound,NA,BJ_NA,2024-04-26 03:53:41,Proposal,contact_3@walmart.com,+1-4536118013,...,2024-04-28 18:11:41,Outbound,CMP_W,High,3,>24 months,50-100L,USA,Finance,Data
2,LDFRD-NA-003,Ford Motor Co Lead,Ford Motor Co,Inbound,NA,CL_NA,2024-04-27 06:06:54,Demo Scheduled,contact_83@fordmotorco.com,+1-9959636818,...,2024-05-01 11:57:54,Inbound,CMP_FMC,Medium,5,12-24 months,20-50L,USA,Technology,Marketing
3,LDAMZ-NA-004,Amazon Lead,Amazon,Outbound,NA,BJ_NA,2024-06-15 20:09:24,Lead Captured,contact_57@amazon.com,+1-7746385663,...,2024-06-17 04:20:24,Outbound,CMP_A,Medium,5,0-6 months,20-50L,USA,Technology,Data
4,LDMST-NA-005,Microsoft Lead,Microsoft,Outbound,NA,BJ_NA,2023-12-12 06:15:25,Demo Completed,contact_15@microsoft.com,+1-6596503700,...,2023-12-13 07:47:25,Outbound,CMP_M,Medium,10,0-6 months,20-50L,USA,Technology,Data


,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Last_Updated,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category
4995,LDSTRA-ME-4888,StrategicIndustries Lead,StrategicIndustries,Outbound,ME,OC_ME,2025-01-25 13:58:40,Negotiation,contact_34@strategicindustries.com,+973-6083148014,...,2025-01-28 12:45:40,Outbound,CMP_S_4888,Low,7,>24 months,100L+,Bahrain,Real Estate,Data
4996,LDINNO-EU-4889,InnovativeIndustries Lead,InnovativeIndustries,Outbound,EU,EB_EU,2023-10-29 18:43:36,Demo,contact_2@innovativeindustries.com,+44-7150916889,...,2023-11-04 07:09:36,Outbound,CMP_I_4889,Low,5,>24 months,100L+,UK,Pharmaceuticals,Consulting
4997,LDADVA-LA-4890,AdvancedConsulting Lead,AdvancedConsulting,Outbound,LA,QH_LA,2024-08-11 22:42:41,Demo Scheduled,contact_100@advancedconsulting.com,+52-0031194698,...,2024-08-16 01:48:41,Outbound,CMP_A_4890,Medium,10,12-24 months,100L+,Mexico,Agriculture,Logistics
4998,LDFUTU-EU-4891,FutureAnalytics Lead,FutureAnalytics,Outbound,EU,HB_EU,2024-06-25 10:55:51,Lead Captured,contact_21@futureanalytics.com,+44-6063960473,...,2024-06-26 11:12:51,Outbound,CMP_F_4891,Low,3,0-6 months,50-100L,UK,Banking,Engineering
4999,LDFUTU-LA-4892,FuturePartners Lead,FuturePartners,Outbound,LA,PY_LA,2025-02-04 19:32:41,Proposal,contact_8@futurepartners.com,+56-0027411837,...,2025-02-06 22:41:41,Outbound,CMP_F_4892,High,3,12-24 months,20-50L,Chile,Agriculture,Operations


In [548]:
# DEALS GENERATION
def generate_deals(df_lead, df_stage_history):
    deals_data = []
    deal_id_counter = 1

    closed_leads = df_lead[df_lead['Current_Stage'].isin(['Deal Won', 'Deal Lost'])]

    for _, lead in closed_leads.iterrows():
        lead_id = lead['Lead_ID']

        lead_captured_ts = df_stage_history[
            (df_stage_history['Lead_ID'] == lead_id) & (df_stage_history['Stage'] == 'Lead Captured')
        ]['Timestamp'].min()

        closed_ts = df_stage_history[
            (df_stage_history['Lead_ID'] == lead_id) & (df_stage_history['Stage'].isin(['Deal Won', 'Deal Lost']))
        ]['Timestamp'].max()

        if pd.notna(lead_captured_ts) and pd.notna(closed_ts):
            total_duration_days = (closed_ts - lead_captured_ts).days if lead['Current_Stage'] == 'Deal Won' else None

            deals_data.append({
                "Deal_ID": f"DEAL_{deal_id_counter:04}",
                "Lead_ID": lead_id,
                "Company_ID": lead['Company_ID'],
                "Amount": round(random.uniform(500, 5000), 2),
                "Status": lead['Current_Stage'],
                "Created_Date": lead_captured_ts.date().strftime("%Y-%m-%d"),
                "Closed_Date": closed_ts.date().strftime("%Y-%m-%d"),
                "Rep_ID": lead['Rep_ID'],
                "Total_Duration_Days": total_duration_days
            })
            deal_id_counter += 1

    return pd.DataFrame(deals_data)

df_deals = generate_deals(df_lead, df_stage_history)
display(df_deals.head())
print(f"Generated {len(df_deals)} deals.")


,Deal_ID,Lead_ID,Company_ID,Amount,Status,Created_Date,Closed_Date,Rep_ID,Total_Duration_Days
0,DEAL_0001,LDKPR-NA-013,CMP_KP,3718.10,Deal Won,2024-10-26,2024-12-05,BJ_NA,40.0
1,DEAL_0002,LDWLF-NA-014,CMP_WF,2059.61,Deal Won,2024-09-01,2024-10-11,DK_NA,40.0
2,DEAL_0003,LDLWS-NA-018,CMP_L,4332.97,Deal Won,2024-07-14,2024-08-25,DK_NA,41.0
3,DEAL_0004,LDTSCO-EU-021,CMP_T,2617.12,Deal Won,2025-04-12,2025-05-21,EB_EU,39.0
4,DEAL_0005,LDLFT-EU-022,CMP_LG,2377.77,Deal Won,2024-11-18,2024-12-24,FG_EU,35.0


Generated 814 deals.


In [549]:
!pip install --upgrade gspread gspread_dataframe

import gspread
from gspread_dataframe import set_with_dataframe
from google.colab import auth
auth.authenticate_user()


In [550]:
# Use the spreadsheet name you created
spreadsheet = client.open("CRM_Capstone")

In [551]:
# Sheet mappings
sheet_mapping = {
    "Leads": df_lead,
    "Sales_Reps": df_sales_rep,
    "Regions": df_region,
    "Stage_History": df_stage_history,
    "Deals":df_deals,
    "Companies": df_company
}

# Write to each sheet
for sheet_name, df in sheet_mapping.items():
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
    except gspread.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=df.shape[0], cols=df.shape[1])

    worksheet.clear()
    set_with_dataframe(worksheet, df)

print("All DataFrames written to Google Sheets.")

All DataFrames written to Google Sheets.


In [552]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
